# Generate Customer Personas



### Overview


Using OpenAI API for GPT-3 genterate customer personas.

Content generation is a hot topic and a viable usecase for Large Language Models LLMs like GPT.


### Free Scripts

This is part of a series of free python scripts. 

You can find all the scripts online at https://github.com/FrontAnalyticsInc/data-winners

Message me on twitter if you have any suggestions.


### About Me

My name is Alton Alexander. I am a Data Science consultant turned entreprenuer building SaaS tools for SEO.

Find more about my free scripts or ask me any question on twitter: @alton_lex


### About Data Winners

Join the conversation:

- private Discord community

- Video tutorials

- Feedback and support on this and other scripts

Join now: https://datawinners.gumroad.com/l/data-analytics-for-seo

# Setup

Steps: 

1) You'll need an API key from OpenAI.

2) Paste the key in the cell below or add it as an environment variable. For best practices see https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety

3) Run the following locally or in a google colab environment

In [27]:
# install libraries
%pip install openai

# load libraries
import openai
from pprint import pprint

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [61]:
# Set your api key
# copy and paste your open api key as a varible

openai.api_key = "sk-PASTEYOURKEYHERE"


# Create a function

Inputs: the prompt

Outputs: the text-generated

In [75]:
def openai_textgen_for_datawinners(
        prompt,
        model = "text-davinci-003",
        max_length = 3000
    ):
    """ A generic wrapper for the endpoint """
    
    response = openai.Completion.create(
      model=model,
      prompt=prompt,
      temperature=0.7,
      max_tokens=max_length,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    
    return response.get('choices',[])[0]['text']

In [93]:

# we build variables
prompt_audience = "data winners"

# we build a prompt template (note the f in front of string!)
prompt = f"""Say how cool this is to all the {prompt_audience}"""

prompt

'Say how cool this is to all the data winners'

In [64]:
# test the function

prompt = "Say how cool this is to all the data winners"
openai_textgen_for_datawinners(prompt)

'\n\nThis is so cool! Congrats to all the data winners! You should be proud of yourselves for all the hard work and dedication you put into your data projects. Your success is an inspiration to us all!'

# Primitive customer idea

Lets say you run a travel website and want to personalize the content for different travelers

## List buyer personas

In [76]:
# describe your target customer generally
target = "B2B buyers who are interested in a saas software tool and program to increase their ecommerce sales"
list_n = 5

prompt = f"""
Example target: customers who are interested in buying a bike

Buyer personas:

1. Parent: A parent who buys bike for child

2. Grandparent: A grandparent who buys bike for grandchild

3. Commuter: A person who buys a bike to ride to work

[Target: {target}]

Please give me {list_n} buyer personas:"""

response = openai_textgen_for_datawinners(prompt)

In [77]:
print(prompt)
print("======")
print(response)


Example target: customers who are interested in buying a bike

Buyer personas:

1. Parent: A parent who buys bike for child

2. Grandparent: A grandparent who buys bike for grandchild

3. Commuter: A person who buys a bike to ride to work

[Target: B2B buyers who are interested in a saas software tool and program to increase their ecommerce sales]

Please give me 5 buyer personas:


1. Small Business Owner: A business owner with a small online store looking for a way to increase sales.

2. Corporate Executive: An executive from a larger company looking for a comprehensive software solution to help increase their ecommerce sales.

3. Digital Marketer: A digital marketer looking for a way to optimize their ecommerce sales process.

4. Web Designer: A web designer looking for a way to create a more user-friendly customer experience for their clients.

5. IT Manager: An IT manager looking for a secure and reliable software solution for their ecommerce needs.


In [79]:
# split the response into a dictionary
personas = {}

for each_response in response.split("\n"):
    try:
        if " - " in each_response.split(".",1)[1]:
            title = each_response.split(".",1)[1].split(" - ")[0].strip()
            description = each_response.split(".",1)[1].split(" - ",1)[1].strip()
        if ":" in each_response.split(".",1)[1]:
            title = each_response.split(".",1)[1].split(":")[0].strip()
            description = each_response.split(".",1)[1].split(":",1)[1].strip()
        personas[ title ] = {
            'title':title,
            'description':description.replace("\n"," ")
        }
        
        print("### ", title)
        print()
        print(personas[title])
        print()
    except:
        True


###  Small Business Owner

{'title': 'Small Business Owner', 'description': 'A business owner with a small online store looking for a way to increase sales.'}

###  Corporate Executive

{'title': 'Corporate Executive', 'description': 'An executive from a larger company looking for a comprehensive software solution to help increase their ecommerce sales.'}

###  Digital Marketer

{'title': 'Digital Marketer', 'description': 'A digital marketer looking for a way to optimize their ecommerce sales process.'}

###  Web Designer

{'title': 'Web Designer', 'description': 'A web designer looking for a way to create a more user-friendly customer experience for their clients.'}

###  IT Manager

{'title': 'IT Manager', 'description': 'An IT manager looking for a secure and reliable software solution for their ecommerce needs.'}



## Build a persona profile

Now we can create a series of prompts to explore each persona, one a time.

In [81]:

def create_profile( persona ):
    
    title = persona['title']
    description = persona['description']
    
    # no name yet but we will create one!
    name = persona.get('name', "nameless")
    gender = persona.get('gender', "unknown")
    groupsize = persona.get('groupsize', None)
    
    questions = {
        "name": f"In one word what is the firstname of the person who is a {title} and who is best described as {description}",
        "gender": f"in one word (male or female) what is the gender of a person named %(name)s who is a {title} and who is best described as {description}",
        "groupsize": f"in one number, give the average size of group where you would find a {title} who is best described as {description}",
        "pain": f"list examples of pains (psychological, spiritul, or physical) felt by a {title} who is best described as {description}",
        "insecurities": f"list what humiliates %(name)s who is a {title} who is best described as {description}",
        "books": f"list what books does %(name)s who is a {title} read if they are best described as {description}",
        "movies": f"list what movies and tv shows does %(name)s watch if they are a {title} and they are best described as {description}",
        "desires": f"list what %(name)s secretly desire as a {title} and especially as it is related to being {description}",
        "socialmedia": f"list which forums, chat rooms, facebook groups, reddit channels, etc that %(name)s frequently visits as a {title} and especially as it is related to being {description}",
        "leadmagents": f"list of lead magnets for captivating {target} especially ones that are a {title} and who are described being {description}"
    }
    
    for qtype in questions:
        
        # create the prompt with special responses
        prompt = questions[qtype] % {"name":name, "gender":gender, "groupsize":groupsize}
        response = openai_textgen_for_datawinners(prompt)
        
        # save the response
        persona[qtype] = response
        
        # special responses
        if qtype == 'name':
            name = response.strip().strip("\n").strip().strip(".")
        if qtype == 'gender':
            gender = response.strip().strip("\n").strip().strip(".")
        if qtype == 'groupsize':
            groupsize = response.strip().strip("\n").strip().strip(".")
            
        print("#### ", qtype)
        print(response)
        print()
        print()

        
# test with one
create_profile( personas[ list(personas.keys())[0] ] )
        

####  name


Entrepreneur


####  gender


Male


####  groupsize


3.5


####  pain


Psychological: 
1. Anxieties about potential business failure 
2. Fear of not being able to keep up with competition 
3. Stress about meeting sales goals 
4. Feelings of isolation due to working alone 

Spiritual: 
1. Doubt in self and in the business 
2. Lack of faith in the future of the business 
3. Feeling of frustration and helplessness 

Physical: 
1. Headaches due to stress 
2. Poor sleep due to worrying 
3. Fatigue due to long work hours 
4. Neck and back pain from sitting at a desk for long periods of time


####  insecurities


1. Not understanding the customer's needs or preferences.
2. Not investing in the right marketing channels to reach the target audience.
3. Not keeping up with the latest trends in e-commerce and online selling.
4. Not having a well-defined product or service offering.
5. Failing to build a strong customer base.
6. Not utilizing social media to its fullest potential.

## Repeat for each of the personas

In [83]:

create_profile( personas[ list(personas.keys())[1]] )

####  name


Edwin


####  gender


Male.


####  groupsize


7.5


####  pain


Psychological Pains:

1. Fear of failure
2. Stress of meeting expectations and goals
3. Anxiety of making the wrong decision with software solutions

Spiritual Pains:

1. Doubt in their ability to make the right decision
2. Feeling of being overwhelmed
3. Loss of direction

Physical Pains:

1. Fatigue
2. Back and shoulder pain
3. Headaches


####  insecurities


1. Being unable to find a comprehensive software solution to help increase their ecommerce sales.
2. Having a team that is unable to work together to find a solution.
3. Having to explain to his superiors why the team was unable to find a solution.
4. Having to admit that their current software is not sufficient to meet their needs.
5. Seeing the company's ecommerce sales decrease while competitors are increasing theirs.


####  books


1. E-Commerce for Dummies by Marsha Collier
2. The E-Commerce Guidebook: From Strategy to Implementation by Alan 

In [56]:
# the third persona
create_profile( personas[ list(personas.keys())[2] )

####  name


Explorer


####  gender


Female.


####  groupsize


7.5


####  pain


Psychological Pains:
- Anxiety
- Stress
- Fear of Failure
- Loneliness

Spiritual Pains:
- Loss of Meaning
- Loss of Purpose
- Feeling Disconnected
- Feeling Unfulfilled

Physical Pains:
- Headaches
- Muscle Tension
- Fatigue
- Insomnia


####  insecurities


1. Not having a plan or a budget for the trip. 
2. Not doing enough research before the trip.
3. Not allowing enough time for planning and preparation.
4. Not taking enough time off from work.
5. Not having a backup plan in case something goes wrong.
6. Not bringing the right clothing or equipment.
7. Not taking enough money for the trip. 
8. Not having a travel insurance policy.
9. Not booking accommodations in advance.
10. Not reading up on the local culture and customs.


####  books


1. Lonely Planet World Guide: A Traveler's Guide to the World's Best Cities and Attractions
2. The Rough Guide to Europe
3. The Complete Guide to Budget Travel:

In [57]:
# fifth persona
create_profile( personas[ list(personas.keys())[4] )

####  name


Explorer


####  gender


Male


####  groupsize


3


####  pain


Psychological:
- Fear of the unknown 
- Anxiety about the safety of their travels 
- Feelings of loneliness and isolation 
- Difficulty adjusting to a new routine and lifestyle

Spiritual:
- Uncertainty about the future 
- Feeling disconnected from their previous life 
- Loss of purpose

Physical:
- Aches and pains due to aging 
- Fatigue from travel 
- Joint pain from long days of sightseeing


####  insecurities


1. Being told you're too old to travel. 
2. Being told you don't have the capacity to learn. 
3. Not being able to keep up with the physical demands of a trip. 
4. Not being able to afford the cost of travel. 
5. Feeling out of place or not belonging in certain places. 
6. Not being able to communicate or understand local customs. 
7. Being ignored or overlooked by other travelers or locals. 
8. Being ridiculed for your lack of knowledge of the culture, history, or language. 
9. Having to ask f

## Initialize your own persona

Let OpenAI create the details

In [86]:
persona_template = {
    "title": "solo traveler",
    "description": "a person who likes to travel on their own to various destinations"
}

create_profile( persona_template )

####  name


Solo


####  gender


Male.


####  groupsize


1


####  pain


1. Loneliness 
2. Homesickness 
3. Feeling of vulnerability 
4. Fear of the unknown 
5. Fatigue 
6. Anxiety 
7. Financial stress 
8. Pressure to make the most of each trip 
9. Difficulty adjusting to new cultures and customs 
10. Difficulty making friends in new places


####  insecurities


1. Being ignored or overlooked by other travelers.
2. Not having someone to share experiences with.
3. Having to pay more for certain activities because of the lack of companions.
4. Feeling uncomfortable in social settings due to not having someone to talk to.
5. Feeling lonely or isolated during the journey.
6. Having to make all travel decisions on one’s own.
7. Having to rely solely on oneself in times of need.
8. Being taken advantage of by locals or other travelers.
9. Not being able to fully immerse oneself in the culture due to limited language knowledge or understanding.
10. Not being able to find a connection wi

In [60]:
persona_template = {
    "title": "travel foodie",
    "description": "a person who likes to try the differnt local cuisine whenever visiting various destinations"
}

create_profile( persona_template )

####  name


Gastronome.


####  gender


Male


####  groupsize


3.5


####  pain


1. Anxiety due to unfamiliarity with local customs and culture
2. Loneliness from being away from friends and family
3. Fear of trying new and exotic dishes
4. Discomfort from digestive issues due to unfamiliar foods
5. Stress from budgeting for each meal
6. Exhaustion from long days of sightseeing and eating
7. Sadness from being away from home for too long
8. Expense of trying multiple restaurants in order to find the best local cuisine


####  insecurities


1. Not being able to try all the local cuisine from a particular destination 
2. Getting sick from eating a local dish 
3. Being served food that is not of very good quality 
4. Not being able to find a restaurant that serves high-quality local cuisine 
5. Not having enough time to try all the different local dishes 
6. Not understanding the local customs and manners when eating out 
7. Being served food that is not authentic to the region 
8. 